In [43]:
import pandas as pd 
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
df2 = pd.read_csv("../data/clean/df_processed.csv")
df2.drop(['Unnamed: 0'],inplace=True,axis=1)

In [4]:
df = df2.copy()

In [8]:
df['readmitted'].replace({'<30':'yes','>30':'yes'},inplace=True)

In [9]:
features = df.drop(['readmitted'],axis=1)
target = df[['readmitted']]

In [10]:
df_cat = df.select_dtypes('object')
df_num = df.select_dtypes('number')


## 1st Try with all data though distribution of target values are not equal

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=3)

In [12]:
# Split the dataset into numerical and categorical column
X_train_cat = X_train.select_dtypes('object')
X_test_cat  = X_test.select_dtypes('object')

X_train_num = X_train.select_dtypes('number')
X_test_num  = X_test.select_dtypes('number')

In [13]:
#Encoding. 


#Decided to use one hot encoding for weight and age  as 
#there's perhaps an ordinal relationship but not well/equally distributed

In [14]:
#first will change binary columns to 0s and 1

# OneHotEncoder with drop='if_binary'
encoder = OneHotEncoder(drop='if_binary', sparse_output=False)

# Identify categorical columns with 2 unique values
binary_cols = [col for col in X_train.columns if X_train[col].nunique() == 2 and X_train[col].dtype == 'object']

# Fit and transform the binary columns
X_train_encoded = X_train.copy()  # Copy the DataFrame to avoid modifying it directly
X_test_encoded = X_test.copy()  # Copy the DataFrame to avoid modifying it directly

# Encode train and test data separately
X_train_encoded[binary_cols] = encoder.fit_transform(X_train[binary_cols])
X_test_encoded[binary_cols] = encoder.transform(X_test[binary_cols])

In [15]:
#using OneHotEncoding for non-binary object columns 

non_binary_cols = [col for col in X_train_encoded.columns if X_train_encoded[col].nunique() > 2 and X_train_encoded[col].dtype == 'object']
non_binary_cols

['race',
 'age',
 'weight',
 'metformin',
 'repaglinide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'pioglitazone',
 'insulin']

In [16]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform the non-binary object columns for X_train_encoded
X_train_encoded_non_binary = encoder.fit_transform(X_train_encoded[non_binary_cols])

# Transform the non-binary object columns for X_test_encoded
X_test_encoded_non_binary = encoder.transform(X_test_encoded[non_binary_cols])

# Get the names of the new columns created by OneHotEncoder
new_columns = encoder.get_feature_names_out(input_features=non_binary_cols)

# Create DataFrames for the encoded non-binary object columns
X_train_encoded_non_binary_df = pd.DataFrame(X_train_encoded_non_binary, columns=new_columns, index=X_train_encoded.index)
X_test_encoded_non_binary_df = pd.DataFrame(X_test_encoded_non_binary, columns=new_columns, index=X_test_encoded.index)

# Concatenate the encoded non-binary object columns with the original DataFrames
X_train_encoded = pd.concat([X_train_encoded.drop(columns=non_binary_cols), X_train_encoded_non_binary_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop(columns=non_binary_cols), X_test_encoded_non_binary_df], axis=1)

In [17]:
X_test.shape

(209, 24)

In [18]:
X_test_encoded.shape

(209, 62)

In [19]:
X_test_encoded.nunique()

gender                 2
time_in_hospital      13
num_lab_procedures    67
num_procedures         7
num_medications       31
                      ..
pioglitazone_up        2
insulin_down           2
insulin_no             2
insulin_steady         2
insulin_up             2
Length: 62, dtype: int64

In [20]:
X_train_encoded.select_dtypes('object').nunique()

Series([], dtype: float64)

In [21]:
normalizer = MinMaxScaler()
normalizer.fit(X_train_encoded)

MinMaxScaler()

In [22]:
X_train_norm_np = normalizer.transform(X_train_encoded)
X_test_norm_np = normalizer.transform(X_test_encoded)

In [23]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns=X_train_encoded.columns, index=X_train_encoded.index)
X_test_norm_df  = pd.DataFrame(X_test_norm_np,  columns=X_test_encoded.columns,  index=X_test_encoded.index)

In [39]:
knn = KNeighborsClassifier(n_neighbors=11)  
knn.fit(X_train_norm_df, y_train)
# Save the model with pickle

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=11)

In [40]:
y_pred = knn.predict(X_test_norm_df)

In [41]:
accuracy = knn.score(X_test_norm_df, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5933014354066986


In [45]:
dt_classifier = DecisionTreeClassifier(random_state=4553)
dt_classifier.fit(X_train_norm_df, y_train)

DecisionTreeClassifier(random_state=4553)

In [48]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_norm_df, y_train)


/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [49]:
# Evaluate Decision Tree Classifier
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- age
- glimepiride
- glipizide
- glyburide
- insulin
- ...
Feature names seen at fit time, yet now missing:
- age_[0-10)
- age_[10-20)
- age_[20-30)
- age_[30-40)
- age_[40-50)
- ...
